In [ ]:
# !pip install datasets==3.6.0
# !pip install transformers
# !pip install tf-keras
# !pip install transformers[torch]
# !pip install wandb
# !pip install librosa
# !pip install numpy==1.26.4

from huggingface_hub import login
login("")

import os
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [ ]:
import logging
import torch
import warnings

logging.basicConfig(level=logging.INFO)
warnings.filterwarnings('ignore')
logging.getLogger("pyngrok").setLevel(logging.ERROR)
logging.getLogger("transformers").setLevel(logging.ERROR)
logging.getLogger("torch").setLevel(logging.ERROR)
logger = logging.getLogger(__name__)

import warnings
warnings.filterwarnings('ignore')

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f'Using device: {device}')
print(f"PyTorch version: {torch.__version__}")
if torch.cuda.is_available():
    print(f"CUDA device: {torch.cuda.get_device_name(0)}")

In [ ]:
from transformers import AutoProcessor, AutoModelForCTC, Trainer, TrainingArguments, DataCollatorWithPadding
from transformers import Wav2Vec2Processor, Wav2Vec2ForCTC
import torch
import librosa
import numpy as np
import re
import gc
from dataclasses import dataclass, field
from typing import Any, Dict, List, Optional, Union
from tqdm import tqdm
import copy

In [ ]:
from datasets import load_dataset, Dataset, Audio
ds = load_dataset("Elormiden/MilaMou_Cypriot_Dataset")

train_ds = ds['train']
eval_ds = ds['validation']

# Load model directly
from transformers import AutoProcessor, AutoModelForCTC

processor = AutoProcessor.from_pretrained("lighteternal/wav2vec2-large-xlsr-53-greek")
model = AutoModelForCTC.from_pretrained("lighteternal/wav2vec2-large-xlsr-53-greek")

In [ ]:
def sampling_map(array):
    sr = array['audio']['sampling_rate']
    tr = 16000
    if sr != tr:
        resample_array = librosa.resample(array['audio']['array'], orig_sr=sr, target_sr=tr)
        array['audio'] = {
            'path': array['audio']['path'],
            'array': resample_array,
            'sampling_rate': tr
        }
    return array


reforged_train = [sampling_map(sample) for sample in tqdm(train_ds, desc="Resampling")]
reforged_eval = [sampling_map(sample) for sample in tqdm(eval_ds, desc="Resampling")]

In [ ]:
def tokens_working_processes_wac2vec2(rf_ds): # Wac2vec2 architecture
    audio_arrays = [sample["audio"]["array"] for sample in rf_ds]
    sentences = [sample["sentence"] for sample in rf_ds]
    pronunciation = [sample["pronunciation"] for sample in rf_ds]

    inputs = processor(
        audio_arrays,
        sampling_rate=16000,
        padding=True,
        max_length=16000,
        truncation=True
    )

    # But it works the opposite way for the light-eternal model, it has to be lowercased

    with processor.as_target_processor():
        labels = processor.tokenizer(
            sentences,
            padding='max_length',
            max_length=512,
            truncation=True
        )

    pronounce_labels = processor.tokenizer(
        pronunciation,
        padding='max_length',
        max_length=512,
        truncation=True
    )

    labels_ids = labels["input_ids"]
    pad_token_id = processor.tokenizer.pad_token_id
    labels_ids = torch.tensor(labels_ids)
    labels_ids[labels_ids == pad_token_id] = -100

    pronounce_labels_ids = pronounce_labels["input_ids"]
    pronounce_labels_ids = torch.tensor(pronounce_labels_ids)
    pronounce_labels_ids[pronounce_labels_ids == 0] = -100

    return {
        **inputs,
        "labels": labels_ids,
        # "pronounce_labels": pronounce_labels_ids # Greek model does not recognize english tokens -> therefore we will skip pronunciation for now
    }


In [ ]:
processed_data_train = tokens_working_processes_wac2vec2(reforged_train)
processed_data_eval = tokens_working_processes_wac2vec2(reforged_eval)
print(processed_data_train['labels'][53])

@dataclass
class DataCollatorCTCWithPadding:
    """
    Data collator that will dynamically pad the inputs received.
    Args:
        processor (:class:`~transformers.Wav2Vec2Processor`)
            The processor used for proccessing the data.
        padding (:obj:`bool`, :obj:`str` or :class:`~transformers.tokenization_utils_base.PaddingStrategy`, `optional`, defaults to :obj:`True`):
            Select a strategy to pad the returned sequences (according to the model's padding side and padding index)
            among:
            * :obj:`True` or :obj:`'longest'`: Pad to the longest sequence in the batch (or no padding if only a single
              sequence if provided).
            * :obj:`'max_length'`: Pad to a maximum length specified with the argument :obj:`max_length` or to the
              maximum acceptable input length for the model if that argument is not provided.
            * :obj:`False` or :obj:`'do_not_pad'` (default): No padding (i.e., can output a batch with sequences of
              different lengths).
        max_length (:obj:`int`, `optional`):
            Maximum length of the ``input_values`` of the returned list and optionally padding length (see above).
        max_length_labels (:obj:`int`, `optional`):
            Maximum length of the ``labels`` returned list and optionally padding length (see above).
        pad_to_multiple_of (:obj:`int`, `optional`):
            If set will pad the sequence to a multiple of the provided value.
            This is especially useful to enable the use of Tensor Cores on NVIDIA hardware with compute capability >=
            7.5 (Volta).
    """

    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need
        # different padding methods
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [{"input_ids": feature["labels"]} for feature in features]

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )
        with self.processor.as_target_processor():
            labels_batch = self.processor.pad(
                label_features,
                padding=self.padding,
                max_length=self.max_length_labels,
                pad_to_multiple_of=self.pad_to_multiple_of_labels,
                return_tensors="pt",
            )

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        batch["labels"] = labels

        return batch

train_hf = Dataset.from_dict(processed_data_train)
eval_hf = Dataset.from_dict(processed_data_eval)


from transformers import TrainingArguments, Trainer

data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


training_args = TrainingArguments(
    output_dir=f"./light-eternal-wac2vec2-54-lr3e-5-batch32/64",
    num_train_epochs=8,

    #################
    per_device_train_batch_size=32,
    per_device_eval_batch_size=64,
    gradient_accumulation_steps=2,
    ################

    learning_rate=3e-5,
    warmup_steps=1500,

    #################### A100
    gradient_checkpointing=True,
    bf16=True,
    dataloader_pin_memory=True,
    dataloader_num_workers=8,
    #################

    save_steps=200,
    eval_steps=50,
    weight_decay=0.01,
    eval_strategy="steps",
    save_strategy="steps",
    load_best_model_at_end=True,
    report_to='wandb',
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    logging_steps=50,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_hf,
    eval_dataset=eval_hf,
    data_collator=data_collator,
    tokenizer=processor.tokenizer,
)


In [ ]:
trainer.train()